In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras 
from keras.models import Sequential
from keras import layers
from tensorflow.keras.utils import to_categorical
from keras import models
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from tensorflow.keras import regularizers

In [2]:
train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
sample_sub = pd.read_csv('../input/tabular-playground-series-may-2022/sample_submission.csv')

EDA and feature selection can be found in my other [notebook](https://www.kaggle.com/code/robertturro/tps-may-2022-eda-feature-selection)

From [Engineering the top three feature interactions](https://www.kaggle.com/competitions/tabular-playground-series-may-2022/discussion/323892)

In [3]:
train['i_02_21'] = (train.f_21 + train.f_02 > 5.2).astype(int) - (train.f_21 + train.f_02 < -5.3).astype(int)
train['i_05_22'] = (train.f_22 + train.f_05 > 5.1).astype(int) - (train.f_22 + train.f_05 < -5.4).astype(int)
i_00_01_26 = train.f_00 + train.f_01 + train.f_26
train['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)
    
test['i_02_21'] = (test.f_21 + test.f_02 > 5.2).astype(int) - (test.f_21 + test.f_02 < -5.3).astype(int)
test['i_05_22'] = (test.f_22 + test.f_05 > 5.1).astype(int) - (test.f_22 + test.f_05 < -5.4).astype(int)
i_00_01_26 = test.f_00 + test.f_01 + test.f_26
test['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)

In [4]:
y_train = train['target']
train = train.drop(['target'],axis=1)

From [TPS May: Decoding f_27](https://www.kaggle.com/code/abhishek123maurya/tps-may-decoding-f-27)

In [5]:
def add_letters_count(data):
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for char in letters:
        data[char] = data['f_27'].str.count(char)
        
    return data

train = add_letters_count(train)
test = add_letters_count(test)
train.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,Q,R,S,T,U,V,W,X,Y,Z
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,0,0,0,0,0,0,0,0,0,0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,0,0,0,0,0,0,0,0,0,0
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,...,0,0,0,0,0,0,0,0,0,0
4,4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def remove_zero(data):
    cross_check = 'UVWXYZ'
    for char in cross_check:
        if data[char].sum() == 0:
            data = data.drop([char], axis=1)
            
    return data

train = remove_zero(train)
test = remove_zero(test)
train.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,K,L,M,N,O,P,Q,R,S,T
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,0,0,0,0,0,0,0,0,0,0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,0,0,0,0,0,0,0,0,0,0
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,...,1,0,0,0,0,0,0,0,0,0
3,3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,...,0,0,0,0,0,0,0,0,0,0
4,4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,...,0,0,0,0,0,0,0,0,0,0


In [7]:
def add_pos(data):
    for i in range(10):
        data['pos' + str(i)] = (data['f_27'].str[i]).apply(lambda x: ord(x)) - 75
        
    return data

train = add_pos(train)
test = add_pos(test)
train.head()

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,pos0,pos1,pos2,pos3,pos4,pos5,pos6,pos7,pos8,pos9
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,-10,-9,-10,-9,-7,-10,-7,-9,-10,-9
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,-10,-8,-10,-8,-8,-10,-7,-8,-6,-9
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,...,-10,-10,-10,-6,-10,-9,-8,0,-10,-7
3,3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,...,-9,-7,-9,-9,-10,-10,-8,-9,-8,-9
4,4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,...,-9,-7,-9,-8,-9,-9,-8,-3,-5,-6


In [8]:
train = train.drop('f_27',axis=1)
test = test.drop('f_27',axis=1)

In [9]:
train = train.set_index('id')
test = test.set_index('id')

train_columns = train.columns
test_columns = test.columns

In [10]:
scaler = StandardScaler()
le = preprocessing.LabelEncoder()

train = scaler.fit_transform(train)
test = scaler.fit_transform(test)

In [11]:
train = pd.DataFrame(data=train,columns=train_columns)
test = pd.DataFrame(data=test,columns=test_columns)

In [12]:
train.head()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,pos0,pos1,pos2,pos3,pos4,pos5,pos6,pos7,pos8,pos9
0,-1.374490,0.237914,-0.244425,0.568674,-0.647037,0.839148,0.113849,-0.622798,1.849207,-0.831915,...,-0.997321,-0.644413,-1.421885,-0.619729,1.243387,-0.937278,0.375802,-1.467209,-1.264420,-0.776771
1,1.699197,-1.712872,-2.230360,-0.544198,1.113558,-1.552654,0.448561,-0.622798,0.592099,0.999917,...,-0.997321,0.022778,-1.421885,0.006711,0.472758,-0.937278,0.375802,-1.291926,1.235714,-0.776771
2,1.683885,0.616078,-1.028335,0.811719,-0.608415,0.113695,-0.707992,-0.622798,-1.293562,-0.221304,...,-0.997321,-1.311605,-1.421885,1.259589,-1.068501,1.066919,-0.228162,0.110336,-1.264420,0.433488
3,-0.118017,-0.589476,-0.805398,2.087827,0.371515,-0.129132,-0.281882,0.584807,-0.036455,-0.831915,...,1.002686,0.689969,0.703292,-0.619729,-1.068501,-0.937278,-0.228162,-1.467209,-0.014353,-0.776771
4,1.150047,-0.177876,-0.665703,-1.099783,0.468368,0.499896,0.408249,0.584807,0.592099,-1.442525,...,1.002686,0.689969,0.703292,0.006711,-0.297871,1.066919,-0.228162,-0.415513,1.860748,1.038617


In [13]:
train.shape

(900000, 63)

In [14]:
model2 = models.Sequential()
model2.add(layers.Dense(750, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation="relu",input_shape=(63,)))
model2.add(layers.BatchNormalization())
model2.add(layers.Dense(512, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation="relu"))
model2.add(layers.Dropout(0.1))
model2.add(layers.Dense(200,kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation="relu"))
model2.add(layers.Dropout(0.2))
model2.add(layers.Dense(60,kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation="relu"))
model2.add(layers.Dropout(0.3))
model2.add(layers.Dense(16,kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation="relu"))
model2.add(layers.Dense(1,activation='sigmoid'))

model2.summary()

2022-05-31 18:52:33.582651: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 750)               48000     
_________________________________________________________________
batch_normalization (BatchNo (None, 750)               3000      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               384512    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               102600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 60)                1

2022-05-31 18:52:33.707522: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 18:52:33.708724: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 18:52:33.710891: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-31 18:52:33.717577: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [15]:
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['AUC'])

In [16]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model2.fit(train, y_train, validation_split=0.3, shuffle=True, epochs=200,batch_size=2000,callbacks=[callback])

2022-05-31 18:52:37.966546: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200
315/315 [==============================] - 6s 11ms/step - loss: 0.5338 - auc: 0.9508 - val_loss: 0.4029 - val_auc: 0.9752
Epoch 2/200
315/315 [==============================] - 2s 8ms/step - loss: 0.3084 - auc: 0.9831 - val_loss: 0.2448 - val_auc: 0.9903
Epoch 3/200
315/315 [==============================] - 2s 8ms/step - loss: 0.2212 - auc: 0.9906 - val_loss: 0.1932 - val_auc: 0.9930
Epoch 4/200
315/315 [==============================] - 2s 8ms/step - loss: 0.1827 - auc: 0.9930 - val_loss: 0.1685 - val_auc: 0.9939
Epoch 5/200
315/315 [==============================] - 3s 8ms/step - loss: 0.1610 - auc: 0.9943 - val_loss: 0.1512 - val_auc: 0.9950
Epoch 6/200
315/315 [==============================] - 3s 8ms/step - loss: 0.1472 - auc: 0.9951 - val_loss: 0.1425 - val_auc: 0.9953
Epoch 7/200
315/315 [==============================] - 2s 8ms/step - loss: 0.1377 - auc: 0.9956 - val_loss: 0.1347 - val_auc: 0.9958
Epoch 8/200
315/315 [==============================] - 3s 9ms/step -

In [17]:
preds = model2.predict(test)

In [18]:
sample_sub['target'] = preds

In [19]:
sample_sub.to_csv('submission.csv', index=False)